In [1]:
from poloniex import Poloniex

polo = Poloniex()

In [2]:
markets = polo.returnTicker()
pairs = []
for name, thinks in markets.items():
    pairs.append(name)

In [3]:
# funciones necesarias para el manejo del tiempo
from datetime import datetime, timedelta
from time import sleep
from tusp import string2ts
from bot import prepareData
import numpy as np
import pylab as pl
%matplotlib inline

class PairData:
    
    def __init__(self, pair, period, weeks = 4.3*3, backto = None):
            
        self.pair = pair
        self.period = period
        delta = timedelta(weeks = weeks)
        self.delta = delta
        start = ""
        end = ""

        # Sí el tiempo final es diferente al actual...
        if backto != None:
            to = datetime.strptime(backto, "%Y-%m-%d %H:%M:%S")
            self.to = to
            start = string2ts(to.strftime('%Y-%m-%d %H:%M:%S'))
            
            tf = to + delta
            self.tf = tf
            end = string2ts(tf.strftime('%Y-%m-%d %H:%M:%S'))
            
        else:
            tf = datetime.now()
            self.tf = tf
            end = string2ts(tf.strftime('%Y-%m-%d %H:%M:%S'))
            
            to = tf-delta
            self.to = to
            start = string2ts(to.strftime('%Y-%m-%d %H:%M:%S'))
                              

        self.start = start
        self.end = end
        df = prepareData(pair=pair, start=start, end=end, period=int(period))
        
        self.df = df
        self.market_return()
        self.cum_volume()
    
    def market_return(self):
        """Calcula el retorno del mercado vía log_retornos
        """
        
        serie = self.df["close"]

        # calculando los log retornos
        log_return = np.log(serie).diff().fillna(0)

        # calculando log retorno acumulado
        cum_logr = log_return.cumsum()
        cum_r = np.exp(cum_logr) - 1
        
        self.l_return = log_return
        self.m_return = cum_r
        
    def cum_volume(self):
        """Calcula el volumen acumulado del mercado
        """
        serie = self.df["volume"]
        self.c_volume = serie.cumsum()[-1]
        
    def sortino(self, log = False):
        """ Calcula en ratio de Sortino
        
        """
        
        if log:
            rets = self.l_return
         
        else:
            rets = self.m_return

        semi_var = rets[rets < 0] ** 2
        semi_var = semi_var.sum()/len(rets)
        sortino = np.sqrt(semi_var)

        return rets.mean()/sortino

    def sharpe(self, period_ret = 0.2):
        """
        Calculate the annualised Sharpe ratio of a returns stream 
        based on a number of trading periods, N. N defaults to 252,
        which then assumes a stream of daily returns.

        The function assumes that the returns are the excess of 
        those compared to a benchmark.
        """
        N = self.delta.days
        returns = self.l_return
        excess_ret = returns - period_ret/N
        return np.sqrt(N) * excess_ret.mean() / excess_ret.std()
    
dgb = PairData("BTC_DGB", 3600*4)

print dgb.c_volume

57114.3603972


In [4]:
# cálculo del ratio de Sortino del último mes
volumes = []
pairs = [x for x in pairs if x.split("_")[0] == "BTC"]
for pair in pairs:
    #print "\n\t",pair
    asset = PairData(pair, 3600*4, 4.28*2)
    volume = asset.c_volume
    volumes.append([pair, volume])
    #print sortino

volumes.sort(key = lambda l:l[1], reverse=True)

In [5]:
print volumes[:6]

[[u'BTC_ETH', 168539.92791192984], [u'BTC_XRP', 140944.53680166011], [u'BTC_LTC', 66937.469443010035], [u'BTC_STR', 64078.616706059976], [u'BTC_XMR', 47776.018147169983], [u'BTC_ETC', 37496.240681529984]]


In [6]:
# cálculo del ratio de Sortino del último mes
sortinos = []
pairs = [x for x in pairs if x.split("_")[0] == "BTC"]
for pair in pairs:
    #print "\n\t",pair
    asset = PairData(pair, 3600*4, 4.28*2)
    sortino = asset.sortino(log = True)
    sortinos.append([pair, sortino])
    #print sortino

sortinos.sort(key = lambda l:l[1], reverse=True)

# Cálculo del ratio de Sortino del penúltimo mes
sortinos2 = []

for pair in pairs:
    #print "\n\t",pair
    asset = PairData(pair, 3600*4, 4.28*2, "2017-11-14 00:00:00")
    sortino = asset.sortino(log = True)
    sortinos2.append([pair, sortino])
    #print sortino

sortinos2.sort(key = lambda l:l[1], reverse=True)

# Cálculo del ratio de Sortino del penúltimo mes
sortinos3 = []

for pair in pairs:
    #print "\n\t",pair
    asset = PairData(pair, 3600*4, 4.28*2, "2017-10-14 00:00:00")
    sortino = asset.sortino(log = True)
    sortinos3.append([pair, sortino])
    #print sortino

sortinos3.sort(key = lambda l:l[1], reverse=True)

In [7]:
print "Octubre a Noviembre"
print sortinos3[:3]
print "Noviembre a Diciembre"
print sortinos2[:3]
print "Diciembre a Enero"
print sortinos[:3]

Octubre a Noviembre
[[u'BTC_NXT', 0.13781739870626794], [u'BTC_EMC2', 0.11688974902141284], [u'BTC_STR', 0.080834759429922021]]
Noviembre a Diciembre
[[u'BTC_STR', 0.1960905926315005], [u'BTC_SC', 0.16516678529416567], [u'BTC_BURST', 0.15996807743790623]]
Diciembre a Enero
[[u'BTC_LTC', 0.027873062895331854], [u'BTC_XPM', 0.0032471204987044614], [u'BTC_XMR', -0.027454726532719031]]


In [8]:
# cálculo del ratio de Sharpe del último mes
sharpes = []
for pair in pairs:
    asset_past = PairData(pair, 3600*4, 4.28*2, "2017-11-14 00:00:00")
    ret = asset_past.m_return[-1]
    asset = PairData(pair, 3600*4, 4.28*2)
    sharpe = asset.sharpe(ret)
    sharpes.append([pair, sharpe])

sharpes.sort(key = lambda l:l[1], reverse=True)

# Cálculo del ratio de Sharpe del penúltimo mes
sharpes2 = []

for pair in pairs:
    asset_past = PairData(pair, 3600*4, 4.28*2, "2017-10-14 00:00:00")
    ret = asset_past.m_return[-1]
    asset = PairData(pair, 3600*4, 4.28*2, "2017-11-14 00:00:00")
    sharpe = asset.sharpe()
    sharpes2.append([pair, sharpe])

sharpes2.sort(key = lambda l:l[1], reverse=True)

# Cálculo del ratio de Sharpe del penúltimo mes
sharpes3 = []

for pair in pairs:
    asset_past = PairData(pair, 3600*4, 4.28*2, "2017-09-14 00:00:00")
    ret = asset_past.m_return[-1]
    asset = PairData(pair, 3600*4, 4.28*2, "2017-10-14 00:00:00")
    sharpe = asset.sharpe()
    sharpes3.append([pair, sharpe])

sharpes3.sort(key = lambda l:l[1], reverse=True)

In [9]:
print "Octubre y Noviembre"
print sharpes3[:3]
print "Noviembre y Diciembre"
print sharpes2[:3]
print "Diciembre y Enero"
print sharpes[:3]

Octubre y Noviembre
[[u'BTC_EMC2', 0.15729527376916327], [u'BTC_NXT', 0.023479674988932761], [u'BTC_STR', -0.075880999224661863]]
Noviembre y Diciembre
[[u'BTC_STR', 0.45279572024813147], [u'BTC_SC', 0.32720856845234464], [u'BTC_BURST', 0.25713595584531107]]
Diciembre y Enero
[[u'BTC_VTC', 1.1657920657433294], [u'BTC_BCH', 0.55443932184412215], [u'BTC_XBC', 0.35182346837883133]]


In [10]:
sortinos3 = [x for x in sortinos if x[0].split("_")[0] == "BTC"]
print sortinos3[0:7]

[[u'BTC_LTC', 0.027873062895331854], [u'BTC_XPM', 0.0032471204987044614], [u'BTC_XMR', -0.027454726532719031], [u'BTC_OMG', -0.030081248023509664], [u'BTC_SYS', -0.031753891875736559], [u'BTC_LSK', -0.036190160208087928], [u'BTC_SBD', -0.04046714412157764]]


In [11]:
pairs = [x for x in pairs if x.split("_")[0] == "BTC"]

In [15]:
from indicators import rsi
rsi_list = []

for pair in pairs:
    #print "\n\t",pair
    asset = PairData(pair, 3600*24, weeks = 4.3*6)
    RSI1, RSI2 = rsi(asset.df["close"])
    rsi_list.append([pair, RSI1[-1], asset.df["close"][-1]])
    #print sortino

rsi_list.sort(key = lambda l:l[1], reverse=False)

print rsi_list[0:6]

[[u'BTC_ETH', 23.764996188576589, 0.062924999999999995], [u'BTC_GNO', 26.922391768002868, 0.009781200000000002], [u'BTC_PPC', 27.195196890499389, 0.00021898999999999999], [u'BTC_BTCD', 27.998199350518306, 0.0083123099999999998], [u'BTC_HUC', 29.678800104551641, 1.7220000000000001e-05], [u'BTC_ZRX', 29.693619341944157, 5.6799999999999998e-05]]


In [14]:
print rsi_list[0:10]

[[u'BTC_ETH', 24.933691131250384, 0.064220029999999997], [u'BTC_NAV', 25.036773210083027, 0.000122], [u'BTC_PPC', 25.384237038423649, 0.00021600999999999999], [u'BTC_VTC', 29.389638294150657, 0.00024623000000000001], [u'BTC_GNO', 29.473609409501393, 0.010110920000000001], [u'BTC_DASH', 29.868725025584794, 0.048166479999999998], [u'BTC_CLAM', 30.324272174623658, 0.0004348], [u'BTC_BTCD', 30.798054804461856, 0.0084408599999999997], [u'BTC_NEOS', 30.841248063790516, 0.00030017999999999999], [u'BTC_ZEC', 31.9386431375993, 0.028821840000000001]]


In [13]:
print rsi_list[-6:]

[[u'BTC_FLO', 45.211004033146033, 1.076e-05], [u'BTC_LTC', 45.460331674891705, 0.018744999999999998], [u'BTC_FCT', 45.752088408571403, 0.0027534], [u'BTC_MAID', 46.393080096353522, 3.6170000000000001e-05], [u'BTC_BCH', 46.835476142559592, 0.11695625], [u'BTC_XPM', 55.580487626291102, 8.7949999999999996e-05]]
